In [1]:
from import_library import *

/Users/monoaki/miniforge3/envs/SCD/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
data_type ='mean' #mat, csv, mix
# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = '/Volumes/DatasetCGRD/DataSet/dFC matrix matlab/' #raw_data_path = './data/image data/dFC matrix matlab/'
# next_path ='mdALFF.csv' #next_path ='C0005FC_z.mat'
class loaders:
    def __init__(self, data_type='mat' ,mat_dtype='FCM_mean'):
        self.data_type = data_type
        if mat_dtype=='mean':
            md ='FCM_mean'
        else:
            md = 'FCM_var'
        self.mat_dtype = md
        self.concat_mat_array = []
    def mat_process(self, data):
        get_data = []
        for idx, i in enumerate(data):
            if len(i[idx+1::])!=0:
                get_data.append(list(i[idx+1::]))
        get_data = list(itertools.chain(*get_data))
        # print([idx for idx, i in enumerate(get_data) if (i>-2)==False])
        return get_data
    def load_mat(self, data_path=None):
        mat = scipy.io.loadmat(data_path)
        get_value = mat[self.mat_dtype ]
        get_value = self.mat_process(get_value)
        self.concat_mat_array.append(get_value)
    def callback(self,):
        return np.array(self.concat_mat_array).astype(np.float16)
ld = loaders(data_type = 'mat', mat_dtype = 'mean')
for next_path in sorted(os.listdir(raw_data_path)):
    if 'C0018FC_z.mat' not in next_path:
        
        ld.load_mat(data_path = os.path.join(raw_data_path,next_path))
patient_value_index = clinical_data['subj']
patient_value = ld.callback()

In [3]:
# model list
def model_list(grid=False, seed=123):
    # svr_param = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')}
    # SVR_ = SVR()
    
    xgbr_param = {'nthread':[4], 'objective':['reg:squarederror'], 'learning_rate': [.03, 0.05, .07], 'max_depth': [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31], 'min_child_weight': [4], 
                                    'subsample': [0.7], 'colsample_bytree': [0.7], 'n_estimators': [400, 450, 500, 550, 600, 660]}
    XGBR_ =XGBRegressor(random_state = seed, verbose=-1)

    lgbmr_param =  {'num_leaves': [7, 14, 21], 'learning_rate': [0.05, 0.005], 'max_depth': [10, 15, 25], 
                                        'min_data_in_leaf':[10, 15, 25], 'feature_fraction': [0.6, 0.8, 0.9],'cat_smooth': [1,10, 15, 20, 35], 'verbose': [-1]}
    
    lgbmr_param =  {'learning_rate': [0.01, 0.05, 0.001,0.005], 'max_depth': [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31], 'verbose': [-1]}
    
    
    LGBMR_ = LGBMRegressor(random_state=seed, verbose=-1)
    
    mlpr_param =  {'hidden_layer_sizes': [(25), (50), (100)]}


    # mlpr_param =  {'hidden_layer_sizes': [(25), (50), (75),(100), (25, 50), (50,100), (25, 50, 75),(50, 75, 100)], 'learning_rate_init': [0.01, 0.03, 0.001, 0.003, 0.0001, 0.0003],
    #                                     'activation': ['relu', 'tanh'], 'solver': ['adam', 'lbfgs'], 'learning_rate' : ['constant', 'adaptive', 'invscaling']}

    MLPR_ = MLPRegressor(activation = 'relu', solver = 'adam', max_iter=20000, random_state=seed, early_stopping=True, validation_fraction=0.50, verbose=0)

    if grid ==True:
        # model_stack = {"SVR":[SVR_, svr_param], "XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
        model_stack = {"MLPR":[MLPR_, mlpr_param], "XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
        # model_stack = {"XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
    else:
        # model_stack = {"SVR":SVR_, "XBGR":XGBR_, "LGBMR": LGBMR_}
        # model_stack = {"XBGR":XGBR_, "LGBMR": LGBMR_}
        model_stack = {"MLPR":MLPR_, "XBGR":XGBR_, "LGBMR": LGBMR_}
    return model_stack

In [7]:
grid =False
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        # classification : AD8 [0-1, >=2]{0,1}, MOCA [<=23, >=24]{0,1}
        # outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
        model_stack = model_list(grid=grid)
        kfold = KFold(n_splits=4)
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['MLPR', 'XBGR', 'LGBMR']:
                    label_ = clinical_data[outcome_name]
                    print("{} Outcome: {}".format(data_type,outcome_name))
                    print("--Start Training {} model--".format(model))
                    if 'MOCA' in outcome_name:
                        label_s = [1 if (i>=24)  else 0  for i in list(label_)]
                    elif 'AD8' in outcome_name:
                        label_s = [1 if i>=2 else 0 for i in list(label_)]
                        label_ = label_+1
                    X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
                    y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)
                    X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), test_size=0.5, random_state=123)
                    y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), test_size=0.5,random_state=123)

                    if grid==False:
                        if model in 'MLPR':
                            model = model_stack[model].fit(X_train, y_train)
                        else:
                            
                            model = model_stack[model].fit(np.array(X_train), np.array(y_train), eval_set=[[np.array(X_valid), np.array(y_valid)]], eval_metric="mape", verbose=False)
                        train_pred =  model.predict(X_train)
                        
                        test_pred =  model.predict(X_valid)
                    else:
                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, n_jobs=-1, verbose=-1, refit = 'neg_mean_squared_error', scoring=['neg_mean_squared_error','neg_mean_absolute_percentage_error'],
                                                 return_train_score=True)
                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold, n_jobs=-1, verbose=-1, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                                                 return_train_score=True)
                        GridModel.fit(np.array(X_train).astype(np.float16), np.array(y_train).astype(np.int8))
                        param_grid = model_stack[model][1]
                        print("Grid Metric", GridModel.cv_results_.keys())
                        for res_ in GridModel.cv_results_:
                            if 'test_score' in res_:
                                print(res_,GridModel.cv_results_[res_])
                        print("Scoring", GridModel.best_score_)
                        print('Best parameter : [{}]\n'.format(GridModel.best_params_))
                        train_pred =  GridModel.predict(X_train)
                        test_pred =  GridModel.predict(X_test)
                        print("Data Length. Training : {} || Validation: {}".format(len(y_train), len(y_valid)))
                        model_stack[model].fit(X_train, y_train, verbose=0)
                        val_pred =  model_stack[model].predict(X_valid)
                        train_pred =  model_stack[model].predict(X_train)

                    print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                    print('Valid MAE: ', round(metrics.mean_absolute_error(y_valid,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_valid,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_valid,test_pred),5))
                    # print('Test MAE: ', round(metrics.mean_absolute_error(y_test,test_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_test,test_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_test,test_pred),5))
                    print('----------------------------------------------------')
                print('\n')
                

mean Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training MLPR model--
Train MAE:  2.70471  ||  MSE:  12.57998  ||  MAPE:  0.8998
Valid MAE:  2.78407  ||  MSE:  12.38481  ||  MAPE:  0.92838
----------------------------------------------------


mean Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training XBGR model--
[14:47:43] WARNING: ../src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Train MAE:  0.00018  ||  MSE:  0.0  ||  MAPE:  0.0001
Valid MAE:  2.06425  ||  MSE:  6.4246  ||  MAPE:  1.11057
----------------------------------------------------


mean Outcome: AD8主觀認知障礙[0-1,>=2]
--Start Training LGBMR model--
Train MAE:  1.85848  ||  MSE:  5.27367  ||  MAPE:  1.03724
Valid MAE:  1.83569  ||  MSE:  4.64871  ||  MAPE:  1

In [5]:
# def fxn():
#     warnings.warn("deprecated", DeprecationWarning)
# if True:
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         fxn()
#         outcome_name = 'MOCA客觀認知測驗分數[<=23,>=24]' #AD8主觀認知障礙[0-1,>=2], MOCA客觀認知測驗分數[<=23,>=24]
#         label_ = clinical_data[outcome_name]

#         model_stack = model_list(grid=True)
#         for model in model_stack:
#             label_ = clinical_data[outcome_name]
#             print("Outcome: {}".format(outcome_name))
#             print("--Start Training {} model--".format(model))
#             if 'MOCA' in outcome_name:
#                 label_s = [1 if (i>=24)  else 0  for i in list(label_)]
#             else:
#                 label_s = [1 if i>=2 else 0 for i in list(label_)]
#             X_train, X_test, Y_label, _  = train_test_split(patient_value, list(label_s) , stratify=list(label_s), random_state=123)
#             y_train, y_test, _, _ = train_test_split(list(label_), list(label_s) , stratify=list(label_s), random_state=123)

#             X_train, X_valid, _, _ = train_test_split(X_train, list(Y_label) , stratify=list(Y_label), random_state=123)
#             y_train, y_valid, _, _ = train_test_split(y_train, list(Y_label) , stratify=list(Y_label), random_state=123)
            
#             print("Data Length. Training : {} || Validation: {}".format(len(y_train), len(y_valid)))

#             model_grid = GridSearchCV(model_stack[model][0], model_stack[model][1], cv = 10, n_jobs = -1, verbose=True)
            
#             model_grid.fit(X_train, y_train, validation_data = (X_valid, y_valid))
#             print('Best parameter : [{}]\n'.format(model_grid.best_params_))
#             val_pred =  model_grid.predict(X_valid)
#             train_pred =  model_grid.predict(X_train)
#             # print([round(y-p, 3) for p, y in zip(train_pred, y_train)])
            
#             # print(val_pred, y_val)
#             print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
#             print('Valid MAE: ', round(metrics.mean_absolute_error(y_valid,val_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_valid,val_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_valid,val_pred),5))
#             print('----------------------------------------------------')
#         print('\n')